In [ ]:

# Efficient Frontier (Binder Ready, Flat Format)
import pandas as pd
import numpy as np
from datetime import datetime, date
import copy
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import objective_functions
import io
import ipywidgets as widgets
from IPython.display import display


In [ ]:

# Upload Excel file
uploader = widgets.FileUpload(accept='.xlsx', multiple=False)
display(uploader)


In [ ]:

# Load DataFrame
if uploader.value:
    file_info = list(uploader.value.values())[0]
    content = io.BytesIO(file_info['content'])
    df = pd.read_excel(content, sheet_name="PyData", engine='openpyxl')
    
    df = df.rename(columns=df.iloc[0]).drop(0)
    df['Dates'] = pd.to_datetime(df['Dates'])
    df = df.set_index("Dates")
    df = df.astype(float)
    df = df[df.index > datetime(2017, 1, 1)]
    print("Data loaded.")
else:
    print("Please upload a valid Excel file.")


In [ ]:

# Select assets
asset_selector = widgets.SelectMultiple(options=df.columns.tolist(), description='Assets:', rows=10)
display(asset_selector)


In [ ]:

# Expected returns input
mu_inputs = {}
boxes = []
for asset in asset_selector.value:
    box = widgets.FloatText(value=0.05, description=asset, step=0.001)
    mu_inputs[asset] = box
    boxes.append(box)
display(widgets.VBox(boxes))


In [ ]:

# Utility functions
def log_returns(df):
    return np.log(df / df.shift(1)).dropna()

def ewma_covariance_matrix(X, alpha):
    T, D = X.shape
    cov = np.zeros((D, D))
    for tt in range(T):
        x_t = X[tt, :]
        if tt == 0:
            cov = x_t[:, np.newaxis] @ x_t[np.newaxis, :]
        else:
            cov = alpha * cov + (1 - alpha) * x_t[:, np.newaxis] @ x_t[np.newaxis, :]
    return cov

def get_single_target_weights(target_return, ef):
    ef_i = copy.deepcopy(ef)
    ef_i.efficient_return(target_return)
    return ef_i.weights

def get_single_target_metrics(target_return, ef):
    ef_i = copy.deepcopy(ef)
    ef_i.efficient_return(target_return)
    return ef_i.portfolio_performance()


In [ ]:

# Run optimizer
df_sel = df[list(asset_selector.value)]
logret = log_returns(df_sel)
cov = ewma_covariance_matrix(logret.values, alpha=0.99)
cov_df = pd.DataFrame(cov * 50, index=df_sel.columns, columns=df_sel.columns)

mu = pd.Series({asset: mu_inputs[asset].value for asset in asset_selector.value})
ef = EfficientFrontier(mu, cov_df, weight_bounds=(0, 1))
ef.add_objective(objective_functions.L2_reg, gamma=0.01)

min_ret, max_ret = ef._max_quadratic_utility_bounds()
return_range = np.linspace(min_ret, max_ret, 25)

weights = [get_single_target_weights(r, ef) for r in return_range]
metrics = [get_single_target_metrics(r, ef) for r in return_range]

weights_df = pd.DataFrame(weights, columns=mu.index)
weights_df["Target Return"] = return_range
metrics_df = pd.DataFrame(metrics, columns=["Return", "Volatility", "Sharpe"])

weights_df.head()


In [ ]:

# Export results
today = date.today().strftime("%d%m%y")
weights_df.to_excel(f"eff_weights_{today}.xlsx", index=False)
metrics_df.to_excel(f"eff_metrics_{today}.xlsx", index=False)
print("Export complete.")
